In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

In [2]:
import pandas as pd

data = pd.read_csv("/kaggle/input/data-combine/data_combined.csv")
data.head()

,Headline,Body ID,Stance,articleBody
0,Police find mass graves with at least '15 bodi...,712,unrelated,Danny Boyle is directing the untitled film\n\n...
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree,Hundreds of Palestinians were evacuated from t...
2,"Christian Bale passes on role of Steve Jobs, a...",137,unrelated,30-year-old Moscow resident was hospitalized w...
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,unrelated,(Reuters) - A Canadian soldier was shot at the...
4,Spider burrowed through tourist's stomach and ...,1923,disagree,"Fear not arachnophobes, the story of Bunbury's..."


In [3]:
X = data.drop(["Stance", "Body ID"], axis = 1)
X.head()

,Headline,articleBody
0,Police find mass graves with at least '15 bodi...,Danny Boyle is directing the untitled film\n\n...
1,Hundreds of Palestinians flee floods in Gaza a...,Hundreds of Palestinians were evacuated from t...
2,"Christian Bale passes on role of Steve Jobs, a...",30-year-old Moscow resident was hospitalized w...
3,HBO and Apple in Talks for $15/Month Apple TV ...,(Reuters) - A Canadian soldier was shot at the...
4,Spider burrowed through tourist's stomach and ...,"Fear not arachnophobes, the story of Bunbury's..."


In [4]:
data['stance_cat'] = data['Stance'].map({'agree':0,'disagree':1,'discuss':2,'unrelated':3}).astype(int)
y = data['stance_cat']
y.head()

0    3
1    0
2    3
3    3
4    1
Name: stance_cat, dtype: int64

In [5]:
y = y.to_numpy()

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((44974, 2), (4998, 2), (44974,), (4998,))

In [12]:
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()

In [13]:
count_vect1 = CountVectorizer()
count_vect2 = CountVectorizer()
X_train_counts_1 = count_vect1.fit_transform(X_train[:, 0])
X_train_counts_1.shape

(44974, 3354)

In [14]:
X_train_counts_2 = count_vect2.fit_transform(X_train[:, 1])
X_train_counts_2.shape

(44974, 23079)

In [15]:
from scipy.sparse import hstack

X_train_counts = hstack([X_train_counts_1, X_train_counts_2]).toarray()
X_train_counts.shape

(44974, 26433)

In [16]:
X_test_counts_1 = count_vect1.transform(X_test[:, 0])
X_test_counts_1.shape

(4998, 3354)

In [17]:
X_test_counts_2 = count_vect2.transform(X_test[:, 1])
X_test_counts_2.shape

(4998, 23079)

In [18]:
X_test_counts = hstack([X_test_counts_1, X_test_counts_2]).toarray()
X_test_counts.shape

(4998, 26433)

In [19]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(44974, 26433)

In [20]:
X_test_tfidf = tfidf_transformer.transform(X_test_counts)
X_test_tfidf.shape

(4998, 26433)

In [21]:
clf = MultinomialNB().fit(X_train_tfidf, y_train.ravel())

In [28]:
X_train[0]

array(["'There is no limit to their depravity' Islamic State FEED mother her own SON",
       'A British man who went to fight against Islamic State in Iraq has said a mother was made to eat her own son after he was kidnapped and murdered by the group.\n\nYasir Abdulla claimed that an elderly Kurdish woman went to Mosul, a city captured by Isis, and demanded to see him.\n\nBut he said she was fed his body.\n\nMr Abdulla, from Keighley, West Yorkshire, told The Sun: ‘She was determined to find her son and went to Isis headquarters and asked to see him.\n\n‘The Isis men told her to sit down because she had travelled a long way and said she should have some food before they took her to meet her son.’\n\nShe was then given meat, rice and soup.\n\nHe continued: ‘But they had killed him and chopped him up and after she finished the meal and asked to see her son they laughed and said “You’ve just eaten him”.’\n\nMr Abdulla decided to go to Iraq after discovering that Isis had come within six 

In [29]:
import numpy as np
example = np.array(["Giant 8ft 9in catfish weighing 19 stone caught in Italy is thought to be the biggest ever reeled in with a rod and line","With its huge, gaping mouth, it looks big enough to swallow a person whole.This giant catfish, which weighed 20 stone (127kg) was caught by twin brothers Dino and Dario Ferrari in the Po River of northern Italy.Dino Ferrari caught the 127 kg catfish in the Po delta of Italy (Sportex Italia)Believed to be one of the largest of its kind ever caught, the creature was pulled from the river on Feb 19 with a rod and line after a battle lasting 40 minutes.The nine foot-long creature, a wels catfish, has been dubbed the monster of the Po by the Italian media.The fish was released after the trophy photos (Sportex Italia)After catching, weighing and photographing it, the brothers released it back into the river, which flows across northern Italy to the Adriatic.It’s a silurus glanis [the Latin name for the species], Dino Ferrari said. The American catfish doesn’t grow to such large dimensions – at most it can weigh 50kg. He estimated the fish to be about 30 years old, based on its size, adding that the species can live for up to 50 years.They don’t range over very large distances, they tend to live in the same stretch of river, moving just a few kilometres either way. They eat all types of fish.To catch them you need a lot of patience but also physical strength. We fought for 40 minutes to reel it in. We tired it out and then lifted it out of the water.The wels catfish, also known as the sheatfish, lives in fresh and brackish water.It is native to eastern Europe and parts of Asia but was introduced to western Europe.The fish prey on other fish, as well as worms, crustaceans, frogs and water fowl.Who knows, maybe we will manage to catch it again in a year’s time, and it will be even more gigantic, said Mr Ferrari"])

test_count1 = count_vect1.transform(example.reshape((2, 1))[0])
test_count2 = count_vect2.transform(example.reshape((2, 1))[1])
test_count1.shape, test_count2.shape

((1, 3354), (1, 23079))

In [ ]:
test_count = hstack([test_count1, test_count2]).toarray()
test_count.shape

In [ ]:
test_tfidf = tfidf_transformer.transform(test_count)
test_tfidf.shape

In [3]:
prediction = clf.predict(test_tfidf)
prediction

NameError: name 'clf' is not defined

In [4]:
y_preds = clf.predict(X_test_tfidf)

NameError: name 'clf' is not defined

In [23]:
y_preds.shape, y_test.shape

((4998,), (4998,))

In [24]:
train_pred = clf.predict(X_train_tfidf)
accuracy_score(y_train.ravel(), train_pred)

NameError: name 'accuracy_score' is not defined

In [25]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test.ravel(), y_preds)

0.7637054821928771

In [26]:
pr = clf.predict(X_train_tfidf)


corr=np.where(pr==y_train.ravel())

In [27]:
pr[20]


0

In [38]:
for x in corr[0]:
    if(pr[x]==2):
        print(x,end=" ")

29 43 64 71 72 75 79 87 106 110 114 115 131 142 157 177 187 200 216 271 291 304 335 362 366 380 393 403 405 413 430 441 445 455 461 471 474 481 484 497 499 505 515 517 521 530 544 555 564 569 590 592 596 602 615 627 656 677 689 695 699 704 733 738 749 756 761 768 781 789 796 821 837 842 848 858 887 905 933 935 944 950 953 967 1009 1019 1028 1032 1046 1053 1056 1073 1075 1084 1094 1109 1110 1113 1122 1131 1172 1175 1182 1194 1197 1209 1210 1218 1233 1251 1267 1297 1302 1305 1314 1333 1357 1386 1404 1406 1416 1426 1434 1440 1444 1461 1485 1494 1502 1520 1537 1557 1568 1573 1583 1587 1590 1593 1596 1610 1616 1617 1622 1628 1629 1641 1652 1656 1669 1672 1676 1683 1686 1702 1721 1722 1740 1746 1748 1765 1772 1778 1781 1788 1799 1816 1830 1832 1839 1843 1853 1855 1872 1882 1895 1903 1910 1921 1922 1940 1964 1970 1975 1979 1982 1993 2015 2023 2035 2041 2055 2082 2086 2092 2103 2106 2109 2123 2131 2136 2138 2153 2172 2184 2186 2225 2232 2272 2284 2294 2306 2311 2333 2334 2356 2361 2376 2381 23

In [41]:
X_train[200]

array(['Meteorite makes big crater in Nicaragua, government says',
       'Stories are coming out of Managua, Nicaragua, that a large bang was heard by residents there Saturday evening, and it registered on seismic detectors as well. A crater was found in the woods measuring something like 12 meters (40 feet) across. Claims are being made that this was a meteorite impact, a space rock a meter or so across slamming into the ground at high speed.\n\nPhil Plait writes Slate’s Bad Astronomy blog and is an astronomer, public speaker, science evangelizer, and author of Death From the Skies!\n\n… but I’m unconvinced. Reports are still a bit sketchy, but my biggest doubts come in two parts:\n\n1) In a city of over two million people, you’d expect a fireball around midnight would’ve been seen by a lot of people. I haven’t heard any confirmed reports of witnesses. Also, I haven’t seen any video from it either. The situation in Chelyabinsk in 2013 was different, of course, but video of that event

In [22]:
import pickle

with open('bayes', 'wb') as files:
    pickle.dump(clf, files)

In [23]:
with open('count_vect1', 'wb') as files:
    pickle.dump(count_vect1, files)
    
with open('count_vect2', 'wb') as files:
    pickle.dump(count_vect2, files)

In [24]:
with open('tfidf_transformer', 'wb') as files:
    pickle.dump(tfidf_transformer, files)